In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage import io
from keras import utils
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers

labels= pd.read_csv("train_master.tsv", sep="\t")
master = pd.read_csv("label_master.tsv", sep="\t")
sample = pd.read_csv("sample_submit.csv", header=None, sep=",")

train_images = []
for fname in labels["file_name"]:
    path = "./train/" + fname
    img = io.imread(path)
    train_images.append(img)
train_images = np.array(train_images)
print(type(train_images), train_images.shape)

test_images = []
for fname in sample[0]:
    path = "./test/" + fname
    img = io.imread(path)
    test_images.append(img)
test_images = np.array(test_images )
print(type(test_images ), test_images.shape)

train_images = train_images / 255
test_images = test_images / 255

y = labels["label_id"]

y_categorical = utils.to_categorical(y)
y_categorical

X_con_image, X_ver_image = np.split(train_images, [40000])
y_con_label, y_ver_label = np.split(y_categorical, [40000])

model = Sequential()

model.add(Conv2D(filters=6, kernel_size=(3,3), padding="same", input_shape=(32,32,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=12, kernel_size=(3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=24, kernel_size=(3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=240))
model.add(Activation("relu"))
model.add(Dense(units=140))
model.add(Activation("relu"))
model.add(Dense(units=40))
model.add(Activation("relu"))
model.add(Dense(units=20))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy",
             optimizer=optimizers.SGD(lr=0.005, momentum=0.9, decay=0.0, nesterov=True))

batch_size=300
epochs=20

history = model.fit(X_con_image, y_con_label,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_ver_image, y_ver_label))

Using TensorFlow backend.


<class 'numpy.ndarray'> (50000, 32, 32, 3)
<class 'numpy.ndarray'> (10000, 32, 32, 3)
Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 11s 267us/step - loss: 2.9748 - val_loss: 2.9292
Epoch 2/20
40000/40000 [==============================] - 9s 237us/step - loss: 2.8722 - val_loss: 2.8345
Epoch 3/20
40000/40000 [==============================] - 10s 248us/step - loss: 2.7267 - val_loss: 2.6368
Epoch 4/20
40000/40000 [==============================] - 10s 255us/step - loss: 2.5777 - val_loss: 2.5498
Epoch 5/20
40000/40000 [==============================] - 10s 252us/step - loss: 2.4858 - val_loss: 2.4232
Epoch 6/20
40000/40000 [==============================] - 10s 249us/step - loss: 2.4099 - val_loss: 2.3769
Epoch 7/20
40000/40000 [==============================] - 10s 255us/step - loss: 2.3542 - val_loss: 2.3515
Epoch 8/20
40000/40000 [==============================] - 10s 248us/step - loss: 2.3022 - val_loss: 2.2965
Epoch 9/2

In [2]:
def learning_plot(history, epochs):
    fig = plt.figure(figsize=(15,5))
    plt.subplot(1,1,1)
    plt.plot(range(1,epochs+1), history.history['loss'])
    plt.plot(range(1,epochs+1), history.history['val_loss'])
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.xticks(range(1,epochs+1))
    plt.ylabel('loss')
    plt.legend(['Construction', 'Verification'], loc='upper right')
    plt.show()
    
learning_plot(history,epochs)

NameError: name 'history' is not defined

In [3]:
print(model.evaluate(X_ver_image ,y_ver_label  ))

10000/10000 [==============================] - 1s 142us/step
1.9876531745910644


In [4]:
pred = model.predict(test_images)

pred = pred.argmax(axis=1)
print(pred)

sample[1] = pred
sample.to_csv("submit6.csv", sep=",", index=None, header=None)

[ 0 16 15 ... 15  8  2]
